# Clustering & Geodemographics

A common challenge in data analysis is how to group observations in a data set together in a way that allows for generalisation: _this_ group of observations are similar to one another, _that_ group is dissimilar to this group. But what defines similarity and difference? There is no _one_ answer to that question and so there are many different ways to cluster data, each of which has strengths and weaknesses that make them more, or less, appropriate in different contexts.

## <span style="color:red;">Important Note for Mac Users</span>

Recent changes in the way that the Mac OS handles the plotting of data means that you need to do certain things in a specific order at the start of any notebook in which you intend to show maps or graphs. Please make a copy of the following code for any notebook that you create and make it the _first_ code that you run in the notebook:

```python
# Needed on a Mac
import matplotlib as mpl
mpl.use('TkAgg')
%matplotlib inline
import matplotlib.pyplot as plt
```

For non-Mac users it _should_ be:

```python
%matplotlib inline
import matplotlib.pyplot as plt
```

This _should_ enable you to create plots, including in the practical that we're about to start! If you forget to run this code then you will probably need to restart the Kernel (Kernel > Restart from the menu).

## Clustering in Python

The most commonly-used _aspatial_ clustering algorighms are all found in [scikit-learn](http://scikit-learn.org/stable/), so that will be the focus of this practical. But just as there are aspatial and spatial statistics, there are also _spatially-aware_ clustering algorithms to be found in [PySAL](http://pysal.readthedocs.io/en/latest/), the Python Spatial Analysis Library.

### Clustering in sklearn

One organisation recently produced a handy scikit-learn cheatsheet that you should [download](https://s3.amazonaws.com/assets.datacamp.com/blog_assets/Scikit_Learn_Cheat_Sheet_Python.pdf). The terminology used in scikit-learn is rather different from anything you will have encountered before (unless you've studied computer science and, possibly, statistics) so it's worth spending a few minutes mapping what you already know on to the sklearn framework:

  | Continuous | Categorical
- | ---------- | -----------
**Supervised** | Regression | Classification
**Unsupervised** | Dimensionality Reduction | Clustering

So clustering is a form of unsupervised (because we don't train the model on what a 'good' result looks like) and categorical (because we get labels out of the model, not predictors) machine learning. Clustering is often used together with PCI (Principal Components Analysis) which is a form of unsupervised dimensionality reduction: data sets with "high dimensionality" are reduced using PCI (you can think of this as a realignment of the axes with the 'data cloud') which has the effect of _maximising the variance_ on each new axis, and the reduced-dimension dataset is then fed to a clustering algorithm. Similarly, supervised approaches are often paired: logistic regression (supervised) is often used with classification (supervised).

Anyway, here's a map to sklearn's algorithms and how to navigate them:

<a href="http://scikit-learn.org/stable/tutorial/machine_learning_map/"><img alt="SciKit-Learn Algorithm Map" src="http://scikit-learn.org/stable/_static/ml_map.png"></a>

### Clustering in PySAL

PySAL is similarly complex and _also_ has a map to help you navigate its complexities -- in this case we're particularly interested in the orange 'branch' of PySAL (labelled clustering!):

![PySAL Map](http://darribas.org/gds_scipy16/content/figs/pysal.png)

### Which Approach is Right?

The reason that there is no 'right' approach (as I said above) is that it all depends on what you're trying to accomplish and how you're _reasoning_ about your problem. The image below highlights the extent to which the different clustering approaches in sklearn can produce different results -- and this is only for the _non-geographic_ algorithms!

<a href="http://scikit-learn.org/stable/modules/clustering.html#clustering"><img alt="Clustering algorithm comparison" src="http://scikit-learn.org/stable/_images/sphx_glr_plot_cluster_comparison_0011.png" /></a>

To think about this in a little more detail:

* If I run an online company and I want to classify my customers on the basis of their product purchases, then I probably don't care much about where they are, only about what they buy. So my clustering approach doesn't need to take geography into account. I might well _discover_ that many of my most valuable customers live in a few areas, but that is a finding, not a factor, in my research.
* Conversely, if I am looking for cancer clusters then I might well care a _lot_ about geography because I want to make sure that I don't overlook a bigger cluster because it's 'hidden' inside an area with lots of non-sufferers. In that case, I want my clusters to take geography into account because I'm looking for agglomerations. That approach might classify an area with a smaller proportion of cancer patients as part of a 'cancer cluster' but that's because it is still significant _because_ of the geography.

So you can undertake a spatial analysis using _either_ approach, it just depends on the role that you think geography should play in producing the clusters in the first place. We'll see this in action today!

## Obtaining Geodemographic Data

For the sake of simplicity we're going to work with roughly the same set of data for London that Alexiou & Singleton used in their _Geodemographic Analysis_ chapter from [Geocomputation: A Practical Primer](https://uk.sagepub.com/en-gb/eur/geocomputation/book241023). Although the implementation in the _Primer_ is in the R programming language, the concerns and the approach are exactly the same.

### NomisWeb

In case you've forgotten, nearly the _entire_ Census is available to download from [InFuse](http://infuse2011.ukdataservice.ac.uk/), but you can often download data 'in bulk' from [NomisWeb](https://www.nomisweb.co.uk/home/detailedstats.asp?resume=no) directly. 

#### Demographic Data

The tables we want are:
* KS102EW: Age structure
* KS201EW: Ethnic group
* KS401EW: Dwellings, household space and accommodation type
* KS402EW: Tenure
* KS403EW: Rooms, bedrooms and central heating
* KS404EW: Car or van availability
* KS501EW: Qualifications and students
* KS603EW: Economic Activity by Sex

To save you the trouble of manually selecting and downloading each table I have assembled everything into a '[Census.zip](https://github.com/kingsgeocomp/applied_gsa/blob/master/data/Census.zip?raw=true)' file. You will want to place this in a directory called `data` but you do _not_ need to unzip it.

#### Geographic Areas

We want London LSOAs, which you can get by specifying 'Select areas within', then '2011 - super output areas - lower layers', and 'region' (leading to London). 

### ONS Boundary Data

Now we need to download the LSOA boundary data. A quick Google search on "2011 LSOA boundaries" will lead you to the [Data.gov.uk portal](https://data.gov.uk/dataset/lower_layer_super_output_area_lsoa_boundaries). The rest is fairly straightforward:
* We want 'generalised' because that means that they've removed some of the detail from the boundaries so the file will load (and render) more quickly.
* We want 'clipped' because that means that the boundaries have been clipped to the edges of the land (e.g. the Thames; the 'Full' data set splits the Thames down the middle between adjacent LSOAs).

**_Note:_** be sure that you select the **2011** data, not the **2001** data.

### Setup

You should drag both the LSOA boundary file and the Census zipfile into a 'data' directory that is the same directory as this notebook so that they're easy to access. You should then:
* Unzip _only_ the LSOA zipfile.
* Rename the directory containing LSOA data to 'lsoa'.

And we're ready to go!

### Other Sources of Data

If you're more interested in US Census data then there's a nice-looking (I haven't used it) [wrapper to the Census API](https://pypi.python.org/pypi/census).

## Getting to Work

To get started we're going to work with pandas and geopandas -- again, nothing new so far but you'll see we've got some new libraries here.

### Specifying the Kernel

**_Note:_** Before you go any further, we need to check that you've got the right 'Kernel' (virutal environment) specified in Jupyter. At top right it should say "Python [spats]" and that is the environment that we want to work in: spats is short Spatial Analysis and that contains all of the libraries that we need for our research. There are other kernels configured and these can be accessed by clicking on the 'Kernel' menu item and then 'Change Kernel'. This feature is well beyond the scope of this practical, but it basically allows you to run multiple 'versions' of Python with different libraries or versions of libraries installed at the same time.

### Importing the Libraries

In [ ]:
import pysal as ps
import numpy as np
import pandas as pd
import geopandas as gpd
import seaborn as sns
import clusterpy as cpy
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import os
import re
import zipfile

import sklearn
sklv = int(sklearn.__version__.replace(".",""))
if sklv < 180:
    print("SciKit-Learn verion is: " + sklearn.__version__)
    print("This notebook relies on a version > 0.18.0")

import random
random.seed(123456789) # For reproducibility

# Make numeric display a bit easier
pd.set_option('display.float_format', lambda x: '{:,.2f}'.format(x))

# Make sure output is into notebook
%matplotlib inline

### Loading the Geodata

Unlike the NS-SeC data this is fairly straightforward using geopandas:

In [ ]:
gdf = gpd.read_file(os.path.join('data','lsoa','Lower_Layer_Super_Output_Areas_December_2011_Generalised_Clipped__Boundaries_in_England_and_Wales.shp'))
print("Shape of LSOA file: {0} rows by {1} columns".format(gdf.shape[0], gdf.shape[1]))
gdf.set_index('lsoa11cd', drop=True, inplace=True)
gdf.head(2)

### Loading the Census Data

You may need to make a few adjustments to the path to get the data loaded on your own computer. But notice what we're now able to do here: using the `zipfile` library we can extract a data file (or any other file) from the Zip archive without even having to open it. Saves even more time _and_ disk space!

In [ ]:
z = zipfile.ZipFile(os.path.join('data','Census.zip'))
z.namelist()

In [ ]:
# We're going to save each data set to a
# separate data frame to make it easier
# to work with during cleaning
d = {}

for r in range(0, len(z.namelist())):
    
    m  = re.search("(?:-)([^\.]+)", z.namelist()[r])
    nm = m.group(1)
    
    print("Processing {0} file: ".format(nm))
    
    if z.namelist()[r] == '99521530-Activity.csv':
        d[nm] = pd.read_csv(z.open(z.namelist()[r]), header=7, skip_blank_lines=True, skipfooter=7, engine='python')
    else: 
        pass
        d[nm] = pd.read_csv(z.open(z.namelist()[r]), header=6, skip_blank_lines=True, skipfooter=7, engine='python')
    
    print("\tShape of dataframe is {0} rows by {1} columns".format(d[nm].shape[0], d[nm].shape[1]))


## Tidying Up

OK, that's a _lot_ of data! 4,835 rows and 88 columns. However, we don't know how much of this is redundant and so need to work out what might need removing from the data set before we can try clustering. So we're going to work our way through each data set in turn so that we can convert them to percentages before combining them into a single, large data set.

### Dwellings

From dewllings we're mainly interested in the housing type since we would expect that housing typologies will be a determinant of the types of people who live in an area. We _could_ look at places with no usual residents as well, or explore the distribution of shared dwellings, but this is a pretty good start.

In [ ]:
t = 'Dwellings'

# If we find this column, this deletes it
if np.where(d[t].columns.values=='2011 super output area - lower layer')[0] >= 0:
    d[t] = d[t].drop('2011 super output area - lower layer', 1)

# List all of the columns    
d[t].columns

In [ ]:
# Select the columns we're interested in analysing
selection = [u'Whole house or bungalow: Detached', u'Whole house or bungalow: Semi-detached',
       u'Whole house or bungalow: Terraced (including end-terrace)',
       u'Flat, maisonette or apartment: Purpose-built block of flats or tenement',
       u'Flat, maisonette or apartment: Part of a converted or shared house (including bed-sits)',
       u'Flat, maisonette or apartment: In a commercial building',
       u'Caravan or other mobile or temporary structure']

# We can't be sure how the totals add up 
# so it's best to generate our own based
# on the selection
d[t]['Total Properties'] = d[t].ix[:, selection].sum(axis=1)

In [ ]:
# Create a new data frame to 
# hold the percentage values
# and initialise it with only
# the 'mnemonic' (i.e. GeoCode)
d_pct = pd.concat(
    [d[t]['mnemonic']], 
    axis=1, 
    keys=['mnemonic'])

# For each of the columns remaining
# in the select
for c in selection:
    m  = re.search("^(?:[^\:]*)(?:\:\s)?(.+)$", c)
    nm = m.group(1)
    if nm == 'e':
        nm = 'Caravan'
    print("Renaming " + c + " to " + nm)
    d_pct[nm] = pd.Series(d[t][c].astype(float)/d[t]['Total Properties'].astype(float))

In [ ]:
d[t + '_pct'] = d_pct
d[t + '_pct'].sample(3)

### Age

Clearly, some areas have more young people, some have older people, and some will be composed of families. A lot of these are going to be tied to 'lifestage' and so will help us to understand something about the types of areas in which they live.

In [ ]:
t = 'Age'

if np.where(d[t].columns.values=='2011 super output area - lower layer')[0] >= 0:
    d[t] = d[t].drop('2011 super output area - lower layer', 1)

d[t].columns

In [ ]:
# Derived columns
d[t]['Age 5 to 14']  = d[t]['Age 5 to 7'] + d[t]['Age 8 to 9'] + d[t]['Age 10 to 14'] 
d[t]['Age 15 to 24'] = d[t]['Age 15'] + d[t]['Age 16 to 17'] + d[t]['Age 18 to 19'] + d[t]['Age 20 to 24']
d[t]['Age 25 to 44'] = d[t]['Age 25 to 29'] + d[t]['Age 30 to 44']
d[t]['Age 45 to 64'] = d[t]['Age 45 to 59'] + d[t]['Age 60 to 64']
d[t]['Age 65+'] = d[t]['Age 65 to 74'] + d[t]['Age 75 to 84'] + d[t]['Age 85 to 89'] + d[t]['Age 90 and over']

# Select the columns we're interested in analysing
selection = ['Age 0 to 4','Age 5 to 14','Age 15 to 24',
             'Age 25 to 44','Age 45 to 64','Age 65+']

# Create a new data frame to 
# hold the percentage values
# and initialise it with only
# the 'mnemonic' (i.e. GeoCode)
d_pct = pd.concat(
    [d[t]['mnemonic']], 
    axis=1, 
    keys=['mnemonic'])

# For each of the columns remaining
# in the select
for c in selection:
    d_pct[c] = pd.Series(d[t][c].astype(float)/d[t]['All usual residents'].astype(float))

d[t + '_pct'] = d_pct
d[t + '_pct'].sample(5)

### Ethnicity

We might also think that the balance of ethnic groups might impact a categorisation of LSOAs in London.

In [ ]:
t = 'Ethnicity'

if np.where(d[t].columns.values=='2011 super output area - lower layer')[0] >= 0:
    d[t] = d[t].drop('2011 super output area - lower layer', 1)

d[t].columns

In [ ]:
# Select the columns we're interested in analysing
selection = ['White', 'Mixed/multiple ethnic groups', 'Asian/Asian British', 
             'Black/African/Caribbean/Black British', 'Other ethnic group']

# Create a new data frame to 
# hold the percentage values
# and initialise it with only
# the 'mnemonic' (i.e. GeoCode)
d_pct = pd.concat(
    [d[t]['mnemonic']], 
    axis=1, 
    keys=['mnemonic'])

# For each of the columns remaining
# in the select
for c in selection:
    d_pct[c] = pd.Series(d[t][c].astype(float)/d[t]['All usual residents'].astype(float))

d[t + '_pct'] = d_pct
d[t + '_pct'].sample(5)

### Rooms

Let's next incorporate the amount of space available to each household.

In [ ]:
t = 'Rooms'

if np.where(d[t].columns.values=='2011 super output area - lower layer')[0] >= 0:
    d[t] = d[t].drop('2011 super output area - lower layer', 1)

d[t].columns

In [ ]:
# Select the columns we're interested in analysing
selection = ['Does not have central heating', 'Occupancy rating (bedrooms) of -1 or less', 
             'Average household size', 'Average number of rooms per household']

# Create a new data frame to 
# hold the percentage values
# and initialise it with only
# the 'mnemonic' (i.e. GeoCode)
d_pct = pd.concat(
    [d[t]['mnemonic']], 
    axis=1, 
    keys=['mnemonic'])

# For each of the columns remaining
# in the select
c = 'Does not have central heating'
d_pct[c] = pd.Series(d[t][c].astype(float)/d[t]['All categories: Type of central heating in household'].astype(float))

c = 'Occupancy rating (bedrooms) of -1 or less'
d_pct[c] = pd.Series(d[t][c].astype(float)/d[t]['All categories: Type of central heating in household'].astype(float))

c = 'Average household size'
d_pct[c] = pd.Series(d[t][c].astype(float))

c = 'Average number of rooms per household'
d_pct[c] = pd.Series(d[t][c].astype(float))

d[t + '_pct'] = d_pct
d[t + '_pct'].sample(5)

### Vehicles

Car ownership and use is also known to be a good predictor of social and economic 'status': Guy Lansley's article on the DLVA's registration database offers a useful perpective on the usefulness of this approach.

In [ ]:
t = 'Vehicles'

if np.where(d[t].columns.values=='2011 super output area - lower layer')[0] >= 0:
    d[t] = d[t].drop('2011 super output area - lower layer', 1)

d[t].columns

In [ ]:
# Select the columns we're interested in analysing
selection = [u'No cars or vans in household', u'1 car or van in household',
       u'2 cars or vans in household', u'3 cars or vans in household',
       u'4 or more cars or vans in household']

# Create a new data frame to 
# hold the percentage values
# and initialise it with only
# the 'mnemonic' (i.e. GeoCode)
d_pct = pd.concat(
    [d[t]['mnemonic']], 
    axis=1, 
    keys=['mnemonic'])

# For each of the columns remaining
# in the select
for c in selection:
    d_pct[c] = pd.Series(d[t][c].astype(float)/d[t]['All categories: Car or van availability'].astype(float))

d[t + '_pct'] = d_pct
d[t + '_pct'].sample(5)

### Tenure

Ownership structure is another categorisation predictor.

In [ ]:
t = 'Tenure'

if np.where(d[t].columns.values=='2011 super output area - lower layer')[0] >= 0:
    d[t] = d[t].drop('2011 super output area - lower layer', 1)

d[t].columns

In [ ]:
# Select the columns we're interested in analysing
selection = [u'Owned', u'Shared ownership (part owned and part rented)', 
             u'Social rented', u'Private rented', u'Living rent free']

# Create a new data frame to 
# hold the percentage values
# and initialise it with only
# the 'mnemonic' (i.e. GeoCode)
d_pct = pd.concat(
    [d[t]['mnemonic']], 
    axis=1, 
    keys=['mnemonic'])

# For each of the columns remaining
# in the select
for c in selection:
    d_pct[c] = pd.Series(d[t][c].astype(float)/d[t]['All households'].astype(float))

d[t + '_pct'] = d_pct
d[t + '_pct'].sample(5)

### Qualifications

In [ ]:
t = 'Qualifications'

if np.where(d[t].columns.values=='2011 super output area - lower layer')[0] >= 0:
    d[t] = d[t].drop('2011 super output area - lower layer', 1)

d[t].columns

In [ ]:
# Select the columns we're interested in analysing
selection = [u'No qualifications',
       u'Highest level of qualification: Level 1 qualifications',
       u'Highest level of qualification: Level 2 qualifications',
       u'Highest level of qualification: Apprenticeship',
       u'Highest level of qualification: Level 3 qualifications',
       u'Highest level of qualification: Level 4 qualifications and above',
       u'Highest level of qualification: Other qualifications']

# Create a new data frame to 
# hold the percentage values
# and initialise it with only
# the 'mnemonic' (i.e. GeoCode)
d_pct = pd.concat(
    [d[t]['mnemonic']], 
    axis=1, 
    keys=['mnemonic'])

# For each of the columns remaining
# in the select
for c in selection:
    d_pct[c] = pd.Series(d[t][c].astype(float)/d[t]['All categories: Highest level of qualification'].astype(float))

d[t + '_pct'] = d_pct
d[t + '_pct'].sample(5)

### Activity

In [ ]:
t = 'Activity'

if np.where(d[t].columns.values=='2011 super output area - lower layer')[0] >= 0:
    d[t] = d[t].drop('2011 super output area - lower layer', 1)

d[t].columns

In [ ]:
# Select the columns we're interested in analysing
selection = [u'Economically active: In employment',
       u'Economically active: Unemployed',
       u'Economically active: Full-time student',
       u'Economically inactive: Retired',
       u'Economically inactive: Looking after home or family',
       u'Economically inactive: Long-term sick or disabled',
       u'Economically inactive: Other']

# Create a new data frame to 
# hold the percentage values
# and initialise it with only
# the 'mnemonic' (i.e. GeoCode)
d_pct = pd.concat(
    [d[t]['mnemonic']], 
    axis=1, 
    keys=['mnemonic'])

# For each of the columns remaining
# in the select
for c in selection:
    m = re.search("^Eco.*?active: (.+)$", c)
    nm = m.group(1)
    d_pct[nm] = pd.Series(d[t][c].astype(float)/d[t]['All usual residents aged 16 to 74'].astype(float))

d[t + '_pct'] = d_pct
d[t + '_pct'].sample(5)

## Bringing it All Together

Now that we've standardised everything using percentages, it's time to bring the data together! We'll initialise the data frame using the first matching data set, and then iterate over the rest, merging the data frames as we go.

In [ ]:
matching = [s for s in d.keys() if "_pct" in s]
print(matching)

lsoac = d[matching[0]]

for m in range(1, len(matching)):
    lsoac = lsoac.merge(d[matching[m]], how='inner', left_on='mnemonic', right_on='mnemonic')

In [ ]:
print(lsoac.columns.values)

In [ ]:
# Change the index
lsoac.set_index('mnemonic', drop=True, inplace=True)
lsoac.index.name = None

In [ ]:
print("Shape of full data frame is {0} by {1}".format(lsoac.shape[0], lsoac.shape[1]))

## Removing Badly-Behaved Variables

Some of these variables will be very difficult to work with because they are so strongly skewed. We can test the degree to which this is a problem fairly easily in pandas:

In [ ]:
if os.path.isdir('outputs') is not True:
    print("Creating 'outputs' directory for images.")
    os.mkdir('outputs')

col_pos=0
for c in lsoac.columns.values:
    print("Creating chart for " + c)
    nm = c.replace("/", "-")
    fig, ax = plt.subplots()
    fig.set_size_inches(7,4)
    sns.distplot(lsoac[c])
    fig.savefig(os.path.join('outputs', "Untransformed-" + str(col_pos) + "." + nm + '.png'))
    plt.close(fig)
    col_pos += 1

In [ ]:
sns.distplot(lsoac.skew(axis=0, numeric_only=True).values).set_title("Skew by Variable for Raw Data")

In [ ]:
sk = lsoac.skew(axis=0, numeric_only=True)
to_drop = sk[sk >= 4].index
print("Dropping highly-skewed variables: " + ", ".join(to_drop.values))

In [ ]:
lsoac.drop(to_drop.values, axis=1, inplace=True)
lsoac.to_pickle(os.path.join('data','LSOAC.pickle'))

## Normalisation

The _Geocomputation_ handbook suggests that normalisation via log or Box-Cox transformation happens _after_ the variables have been converted to percentages, so that's what I've done here. I think that this approach is debatable as it's potentially harder to deal with zeroes in the data _after_ converting to a percentage than it was before. The reason that zeroes are an issue is that the log of 0.0 is `-Inf` or `NaN`, so this blows up in your cluster analysis if you don't deal with it now. The easiest way to do this is to simply add `1` to every raw count, ensuring that the smallest value in your data set is always positive. If you had already converted to a percentage then adding 0.000001% to only the zero values still changes the actual distribution, while adding 0.000001% to all values could leave you with percentages over 100!

I will try to investigate this further when I have time...

In [ ]:
lsoac = pd.read_pickle(os.path.join('data','LSOAC.pickle'))
numeric_cols = [col for col in lsoac if lsoac[col].dtype.kind != 'O']
lsoac[numeric_cols] += 1
print("Numeric columns: " + ", ".join(numeric_cols))

In [ ]:
lsoac['Does not have central heating'].min()

In [ ]:
from scipy.stats import boxcox

col_pos = 0
for c in lsoac.columns:
    if lsoac[c].dtype.kind != 'O':
        print("Transforming " + c)
        x, _ = boxcox( lsoac[c] )
        nm = c.replace("/", "-")
        fig, ax = plt.subplots()
        fig.set_size_inches(7,4)
        sns.distplot(x, hist=True)
        fig.savefig(os.path.join('outputs', "Box-Cox-" + str(col_pos) + "." + nm + '.png'))
        plt.close(fig)
        col_pos += 1
        lsoac[c] = pd.Series(x, index=lsoac.index)

## Removing Correlated Variables

We don't want to keep too many correlated variables in the clustering data since that will bias the clustering algorithms and may result in poor 'performance' in terms of cluster quality (it will be equivalent to some features getting double-weighted!). The best way to do this is to produce a correlation table for all variables and then look to remove problematic variables.

In [ ]:
lsoac.corr()

In [ ]:
corrs    = 0.50 # What's our threshold for strongly correlated?
corrh    = 0.70 # What's our threshold for highly correlated?

# Generate the matrix but capture the output this time
corrm = lsoac.corr()
corrm['name'] = corrm.index # We need a copy of the index

num_corrs = []
hi_corrs  = []

for c in corrm.columns:
    if c != 'name':
        hits = corrm.loc[(corrm[c] >= corrs) & (corrm[c] < 1.0), c]
        print("=" * 20 + " " + c + " " + "=" * 20)
        print("Strongly correlated with : ")
        if hits.size > 0: 
            print("\t" + ", ".join(hits.index.values))
            num_corrs.append(hits.size)
            
            if hits[ hits > corrh ].size > 1:
                print("Highly correlated with: ")
                print("\t" + ", ".join(hits[ hits > corrh ].index.values))
                hi_corrs.append(hits[ hits > corrh ].size)

In [ ]:
sns.distplot(num_corrs, bins=range(1,20), kde=False).set_title("Number of Strong (> " + str(corrs) + ") Correlations with Other Variables")

In [ ]:
sns.distplot(hi_corrs, bins=range(0,20), kde=False).set_title("Number of Very Strong Correlations (> " + str(corrh) + ")  with Other Variables")

### Stripping Out 'Redundant' Variables

Let's remove any variable that has either:
1. A '*lot*' of correlations in excess of 0.50, though we need to define what is 'a lot'.
2. A correlation higher than 0.70 with at least one other variable that is already in our 'to keep list.

This will reduce the dimensionality of our data and allow us to then focus on normalisation.

In [ ]:
maxcorrs = 3 # What's our threshold for too many strong correlations?

to_drop = [] # Columns to drop
to_keep = [] # Columns to keep

for c in corrm.columns:
    if c != 'name':
        
        hits = corrm.loc[(corrm[c] >= corrs) & (corrm[c] < 1.0), c]
        
        print("=" * 12 + " " + c + " " + "=" * 12)
        print(hits)
        print(" ")
        
        hi_vals    = False
        multi_vals = False
        
        # Remove ones with very high correlations
        if hits[ hits > corrh ].size > 0:
            print(">>> Very high correlation...")
            s1 = set(to_keep)
            s2 = set(hits[ hits > corrh ].index.values)
            #print("Comparing to_keep(" + ", ".join(s1) + ") to hits(" + ", ".join(s2) + ")")
            s1 &= s2
            #print("Column found in 'very high correlations': " + str(s1))
            if len(s1) > 1: 
                hi_vals = True
                print("Will drop '" + c + "' because of very high correlation with retained cols: \n\t" + "\n\t".join(s1))
        
        # Remove ones with many correlations
        if hits.size >= maxcorrs: 
            print(">>> Many correlations...")
            s1 = set(to_keep)
            s2 = set(hits.index.values)
            #print("Comparing to_keep(" + ", ".join(s1) + ") to hits(" + ", ".join(s2) + ")")
            s1 &= s2
            #print("Column found in 'many correlations' :" + str(s1))
            if len(s1) > 1: 
                multi_vals = True
                print("Will drop '" + c + "' because of multiple strong correlations with retained cols: \n\t" + "\n\t".join(s1))
        
        if hi_vals==True or multi_vals==True:
            to_drop.append(c)
        else:
            to_keep.append(c)

print(" ")
print("To drop: " + ", ".join(to_drop))
print(" ")
print("To keep: " + ", ".join(to_keep))

In [ ]:
lsoacf = lsoac.drop(to_drop, axis=1, errors='raise')
print("Retained variables: " + ", ".join(lsoacf.columns.values))

## Standardising the Data

The effectiveness of clustering algorithms is usually demonstrated using the 'iris data' -- it's available by default with both Seaborn and SciKit-Learn. This data doesn't usually need normalisation but it's a good way to start looking at the data across four dimensions and seeing how it varies and why some dimensions are 'good' for clustering, while others are 'not useful'...

In [ ]:
sns.set()
irises = sns.load_dataset("iris")
sns.pairplot(irises, hue="species")

### Why Standardise

One of the main challenges of clustering, however, is that the scale of each dimension matters: if you were to try to cluster, for example, [1] how many metres per year a glacier moved with [2] the number of cubic metres by which it grew, then you would only be clustering on variable [2]. 

That's because glaciers contain millions of cubic metres of ice and will grow or shrink by thousands of cubic metres each year. In contrast, most glaciers move at most a few metres per year. So the sheer scale difference between these two dimensions means that the values of variable 1 dominate the clustering algorithm because they provide a much better 'spread' in the data than variable 2.

To address this we need to standardise the data in some way so that the scales are relatively consistent. There's no _one_ way to standardise the data, it depends on the characteristics of the data _as well as_ what we're looking for in terms of clustering. As a general rule, we're _aiming_ for a normal (a.k.a. Gaussian) distribution with 0 mean and unit variance. The latter part of this is what most people focus on: you may recall our work with transformations last year, and here's one more reason why it's useful. That said, normalisation (making the data 'look' normal) can also be very important for the process since it can significantly skew the results as well if the data itself is heavily skewed.

Right, so does this all sound a little bit familiar from last year? Let's start by just looking at a few variables in a simple scatter plot...

In [ ]:
# The data as it is now...
sns.set(style="whitegrid")
sns.pairplot(lsoacf, 
             vars=['Does not have central heating','Owned','Highest level of qualification: Level 4 qualifications and above'], 
             markers=".", size=4, diag_kind='kde')

So there are clearly some differences, but I'd be hard-pressed to give you sensible clusters just by looking at this data. In particular, we have a _real_ problem with the comparisons since 'No Central Heating' _looks_ like it has a very compressed distribution... If we have this problem of comparing different scales, so does the computer.

### Standardisation with SKLearn

Let's try standardising the data now:

In [ ]:
# Here's how we can rescale in a robust way
from sklearn import preprocessing

In [ ]:
plt.rcParams['figure.figsize']=(7,3)
sns.distplot(lsoacf['Does not have central heating'])

In [ ]:
sns.distplot(
    preprocessing.minmax_scale(lsoacf['Does not have central heating'].values.reshape(-1,1), feature_range=(-1.0, 1.0)))

In [ ]:
# Full copy, not copy by reference
df = lsoacf.copy(deep=True)

# An alternative if you'd like to try it
#scaler = preprocessing.RobustScaler(quantile_range=[5.0, 95.0])
scaler = preprocessing.MinMaxScaler(feature_range=(-1, 1))

df[to_keep] = scaler.fit_transform(df[to_keep])

In [ ]:
# The data as it is now...
sns.pairplot(df, 
             vars=['Does not have central heating','Owned','Highest level of qualification: Level 4 qualifications and above'], 
             markers=".", size=4, diag_kind='kde')

Right, so you can see that robustly rescaling the dimension hasn't fundamentally changed the data itself, but it has changed it so that the the data it broadly re-centered on 0 and yet we still have the original outliers from the raw data. You could _also_ do IQR standardisation (0.25 and 0.75) with the percentages, but in those cases you would have _more_ outliers and then _more_ extreme values skewing the results of the clustering algorithm.

In [ ]:
df.describe()

### A Step Too Far?

The standardisation process has given us a better perspective on _where_ high concentrations of different groups might be found, but we still need to decide whether the clustering or other machine learning processes should be influenced by the full range of the data. I followed the approach outlined in _Geocomputation_, but in some ways I lean towards _not_ completely rescaling on the basis that super-high concentrations of particular groups _should_ have a significant impact on the results of the clustering process; however, using robust rescaling (allowing outliers to persist) does mean that we're more likely to get one large cluster containing the bulk of the non-extreme data and a number of small clusters each containing a small number of 'extreme' LSOAs. **Can you think why?**

My point is that the right choice is the one that you can argue logically and consistently for. There are plenty of researchers who would disagree with me on the paragraph above, but that doesn't mean I'm wrong. Nor does it mean they're wrong.

## Let's Cluster!

OK, we're finally here! It's time to cluster the cleaned, normalised, and standardised data set! We're going to start with the best-know clustering technique and work from there...

### K-Means

In [ ]:
from sklearn.cluster import KMeans
#help(KMeans)
df.head(3)

The next few code blocks may take a while to complete, largely because of the `pairplot` at the end where we ask Seaborn to plot every dimension against every other dimension _while_ colouring the points according to their cluster. I've reduced the plotting to just three dimensions, if you want to plot all of them, then just replace the array attached to `vars` with `main_cols`, but you have to bear in mind that that is plotting 4,300 points _each_ time it draws a plot... and there are 81 of them! It'll take a while, but it _will_ do it, and try doing that in Excel or SPSS?

In [ ]:
df.columns.values

In [ ]:
# Quick sanity check in case something hasn't
# run successfully -- these muck up k-means
df.drop(list(df.columns[df.isnull().any().values].values), axis=1, inplace=True)

k      = 7 # Number of clusters
k_var  = 'KMeans' # Variable name
kmeans = KMeans(n_clusters=k).fit(df) # The process

print(kmeans.labels_) # The results

# Add it to the data frame
df[k_var] = pd.Series(kmeans.labels_, index=df.index) 

# How are the clusters distributed?
df.KMMain.hist(bins=k)

# Going to be a bit hard to read if 
# we plot every variable against every
# other variables, so we'll just pick a 
# few
sns.pairplot(df, 
             vars=['Age 0 to 4','Asian/Asian British','In employment'], 
             hue=k_var, markers=".", size=3, diag_kind='kde')

### One More Thing...

There's just _one_ little problem: what assumption did I make when I started this _k_-means cluster analysis? It's a huge one, and it's one of the reasons that _k_-means clustering _can_ be problematic when used naively...

**_Discuss the answer to this question with your neighbour._**

### The Silhouette Plot vs Within-Cluster Sum of Squares

Again, there's more than one way to skin this cat. In _Geocomputation_ they use WCSS to pick the 'optimal' number of clusters. The idea is that you plot the average WCSS for each number of possible clusters in the range of interest (`2...n`) and then look for a 'knee' (i.e. kink) in the curve. The principle of this approach is that you look for the point where there is declining benefit from adding more clusters. The problem is that there is always _some_ benefit to adding more clusters (the perfect clustering is _k==n_), so you don't always see a knee. 

Another way to try to make the process of selecting the number of clusters a little less arbitrary is called the silhouette plot and (like WCSS) it allows us to evaluate the 'quality' of the clustering outcome by examining the distance between each observation and the rest of the cluster. In this case it's based on Partitioning Around the Medoid (PAM). 

Either way, to evaluate this in a systematic way, we want to do _multiple_ _k_-means clusterings for _multiple_ values of _k_ and then we can look at which gives the best results...

Let's try it for the range 3-9.

In [ ]:
# Adapted from: http://scikit-learn.org/stable/auto_examples/cluster/plot_kmeans_silhouette_analysis.html
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score

for k in range(3,10):
    # Debugging
    print("Cluster count: " + str(k))
    
    #############
    # Do the clustering using the main columns
    clusterer = KMeans(n_clusters=k, random_state=10)
    cluster_labels = clusterer.fit_predict(df)
    
    # Calculate the overall silhouette score
    silhouette_avg = silhouette_score(df, cluster_labels)
    print("For k =", k,
          "The average silhouette_score is :", silhouette_avg)
    
    # Calculate the silhouette values
    sample_silhouette_values = silhouette_samples(df, cluster_labels)
    
    #############
    # Create a subplot with 1 row and 2 columns
    fig, (ax1, ax2) = plt.subplots(1, 2)
    fig.set_size_inches(9, 5)

    # The 1st subplot is the silhouette plot
    # The silhouette coefficient can range from -1, 1
    ax1.set_xlim([-1.0, 1.0]) # Changed from -0.1, 1
    
    # The (n_clusters+1)*10 is for inserting blank space between silhouette
    # plots of individual clusters, to demarcate them clearly.
    ax1.set_ylim([0, df.shape[0] + (k + 1) * 10])
    
    y_lower = 10
    
    # For each of the clusters...
    for i in range(k):
        # Aggregate the silhouette scores for samples belonging to
        # cluster i, and sort them
        ith_cluster_silhouette_values = \
            sample_silhouette_values[cluster_labels == i]

        ith_cluster_silhouette_values.sort()

        size_cluster_i = ith_cluster_silhouette_values.shape[0]
        y_upper = y_lower + size_cluster_i
        
        # Set the color ramp
        color = cm.spectral(float(i) / k)
        ax1.fill_betweenx(np.arange(y_lower, y_upper),
                          0, ith_cluster_silhouette_values,
                          facecolor=color, edgecolor=color, alpha=0.7)

        # Label the silhouette plots with their cluster numbers at the middle
        ax1.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))

        # Compute the new y_lower for next plot
        y_lower = y_upper + 10  # 10 for the 0 samples

    ax1.set_title("The silhouette plot for the clusters.")
    ax1.set_xlabel("The silhouette coefficient values")
    ax1.set_ylabel("Cluster label")

    # The vertical line for average silhouette score of all the values
    ax1.axvline(x=silhouette_avg, color="red", linestyle="--")

    ax1.set_yticks([])  # Clear the yaxis labels / ticks
    ax1.set_xticks(range(-1.0, 1.0, 0.2)) # Was: [-0.1, 0, 0.2, 0.4, 0.6, 0.8, 1]

    # 2nd Plot showing the actual clusters formed --
    # we can only do this for the first two dimensions
    # so we may not see fully what is causing the 
    # resulting assignment
    colors = cm.spectral(cluster_labels.astype(float) / k)
    ax2.scatter(df[df.columns[0]], df[df.columns[1]], marker='.', s=30, lw=0, alpha=0.7,
                c=colors)

    # Labeling the clusters
    centers = clusterer.cluster_centers_
    
    # Draw white circles at cluster centers
    ax2.scatter(centers[:, 0], centers[:, 1],
                marker='o', c="white", alpha=1, s=200)

    for i, c in enumerate(centers):
        ax2.scatter(c[0], c[1], marker='$%d$' % i, alpha=1, s=50)

    ax2.set_title("Visualization of the clustered data")
    ax2.set_xlabel("Feature space for the 1st feature")
    ax2.set_ylabel("Feature space for the 2nd feature")

    plt.suptitle(("Silhouette analysis for KMeans clustering on sample data "
                  "with n_clusters = %d" % k),
                 fontsize=14, fontweight='bold')

    plt.show()

### Interpreting the Results

When I ran _k_-means, the results suggested that 7 clusters was probably 'best' -- but note that that's only if we don't have any kind of underlying _theory_, other _empirical evidence_, or just a _reason_ for choosing a different value... Again, we're now getting in areas where _your judgement_ and your ability to _communicate your rationale_ to readers is the key thing. 

Let's repeat the 7-cluster process and then map it.

In [ ]:
# Quick sanity check in case something hasn't
# run successfully -- these muck up k-means
df.drop(list(df.columns[df.isnull().any().values].values), axis=1, inplace=True)

k_pref = 7
kmeans = KMeans(n_clusters=k_pref).fit(df)
df[k_var] = pd.Series(kmeans.labels_, index=df.index)

sdf = gdf.join(df, how='inner')

from pysal.contrib.viz import mapping as maps

# Where will our shapefile be stored
shp_link = os.path.join('data','lsoas.shp')

# Save it!
sdf.to_file(shp_link)

# And now re-load the values from the DBF file 
# associated with the shapefile.
values = np.array(ps.open(shp_link.replace('.shp','.dbf')).by_col(k_var))

maps.plot_choropleth(shp_link, values, 'unique_values', 
                     title='K-Means ' + str(k_pref) + ' Cluster Analysis', 
                     savein=os.path.join('outputs', 'K-Means.png'), dpi=150, 
                     figsize=(8,6), alpha=0.9
                    )

To make sense of whether this is a 'good' result, you might want to visit [datashine](http://datashine.org.uk/#table=QS607EW&col=QS607EW0050&ramp=RdYlGn&layers=BTTT&zoom=10&lon=-0.1751&lat=51.4863) or think back to last year when we examined the NS-SeC data. 

You could also think of ways of plotting how these groups differ. For instance...

In [ ]:
for kl in range(0,k_pref):
    print("Cluster " + str(kl))
    lsoas = df[df[k_var]==kl].index
    print("\t{0}".format(len(lsoas)))
    fig, ax = plt.subplots()
    fig.set_size_inches(9,6)
    for c in to_keep:
        sns.distplot(lsoac[lsoac.index.isin(lsoas.values)][c], hist=False, kde=True, axlabel='')
    fig.savefig(os.path.join('outputs', k_var + "Cluster-" + str(kl) + '.png'))
    plt.close(fig)
print("Done.")

### _Really, really important _

Now would be a good time to think about _how_ standardisation and normalisation would have changed your results... and you might want to test whether applying these in a 'stronger' format (e.g. sklearn's `robust_rescale` and scipy's `boxcox`) help or hinder your analysis!

In [ ]:
df.to_pickle(os.path.join("outputs","clusters.pickle"))

## DBScan

Of course, as we've said above _k_-means is just one way of clustering, DBScan is another. Unlike _k_-means, we don't need to specify the number of clusters in advance. Which sounds great, but we still need to specify _other_ parameters and these can have a huge impact on our results!

In [ ]:
d_var = 'DBSCAN'

# Quick sanity check in case something hasn't
# run successfully -- these muck up k-means
df.drop(list(df.columns[df.isnull().any().values].values), axis=1, inplace=True)

from sklearn.cluster import DBSCAN

# Run the clustering
dbs = DBSCAN(eps=1, min_samples=10).fit(df.as_matrix())

# See how we did
df[d_var] = pd.Series(dbs.labels_, index=df.index)
print(df[d_var]value_counts())

Now that we've got the clustering results we can join it to the Shapefile data (from the geopandas dataframe) and save  it as a new Shapefile containing the new columns.

In [ ]:
sdf = gdf.join(df, how='inner')

sdf.sample(3)[[k_var,d_var]]

And create a new visualisation with the output saved to a PNG file.

In [ ]:
from pysal.contrib.viz import mapping as maps

# Where will our shapefile be stored
shp_link = os.path.join('data','lsoas.shp')

# Save it!
sdf.to_file(shp_link)

# And now re-load the values from the DBF file 
# associated with the shapefile.
values = np.array(ps.open(shp_link.replace('.shp','.dbf')).by_col(d_var))

maps.plot_choropleth(shp_link, values, 'unique_values', 
                     title='DBSCAN Cluster Analysis', 
                     savein=os.path.join('outputs', d_var + '.png'), dpi=150, 
                     figsize=(8,6), alpha=0.9
                    )

### Note!

Note that you now have the results of the clustering process saved in a shapefile and could then load it into a GIS. Or you could also save the `df` file to a pickle for futher analysis...

In [ ]:
df.to_pickle(os.path.join("outputs","clusters.pickle"))

## Self-Organising Maps

SOMs offer a third type of clustering algorithm. They are a relatively 'simple' type of neural network in which the 'map' (of the SOM) adjusts to the data: we're going to see how this works over the next few code blocks, but the main thing is that, unlike the above approaches, SOMs build a 2D map of a higher-dimensional space and use this as a mechanism for subsequently clustering the raw data.

Here's how to install the SOM library that we'll be using:

```shell
git clone https://github.com/sevamoo/SOMPY.git
cd SOMPY
source activate spats
pip setup.py install
```

This should not show any errors -- if it does, try deleting the SOMPY directory and starting again.

In [ ]:
%matplotlib inline
import numpy as np
from matplotlib import pyplot as plt

Where to store the cols:

In [ ]:
s_var = 'SOM'

df = pd.read_pickle(os.path.join("outputs","clusters.pickle"))
from sompy.sompy import SOMFactory
df.sample(2)

Setting up the inputs...

In [ ]:
data  = df.drop([k_var,d_var], axis=1).as_matrix()
names = df.columns.values
print(data[1:2,])

Next, we are going to actually train the SOM using the input data. This is where you specify the input parameters that have the main effect on the clustering results:

In [ ]:
sm = SOMFactory().build(
    data, mapsize=(10,15),
    normalization='var', initialization='random', component_names=names)
sm.train(n_job=4, verbose=False, train_rough_len=2, train_finetune_len=5)

How good is the fit?

In [ ]:
topographic_error  = sm.calculate_topographic_error()
quantization_error = np.mean(sm._bmu[1])
print "Topographic error = %s; Quantization error = %s" % (topographic_error, quantization_error)

How do the results look?

In [ ]:
from sompy.visualization.mapview import View2D
view2D = View2D(10, 10, "rand data", text_size=10)
view2D.show(sm, col_sz=4, which_dim="all", desnormalize=True, cmap='plasma')

How many clusters do we want and where are they on the map?

In [ ]:
k_val = 5

from sompy.visualization.hitmap import HitMapView

sm.cluster(k_val)
hits  = HitMapView(7, 7, "Clustering", text_size=9, cmap='Blues')
a     = hits.show(sm)

How many data points were assigned to each BMU?

In [ ]:
How many data points were assigned to each BMU

In [ ]:
from sompy.visualization.bmuhits import BmuHitsView
vhts = BmuHitsView(5, 5, "Hits Map", text_size=11)
vhts.show(sm, anotate=True, onlyzeros=False, labelsize=9, cmap="plasma", logaritmic=False)

Finally, let's get the cluster results and map them back on to the data points:

In [ ]:
# Get the labels for each BMU
# in the SOM (15 * 10 neurons)
clabs = sm.cluster_labels

# Project the data on to the SOM
# so that we get the BMU for each
# of the original data points
bmus  = sm.project_data(data)

# Turn the BMUs into cluster labels
# and append to the data frame
df[s_var] = pd.Series(clabs[bmus], index=df.index)

print(df.SOM.value_counts())

sdf = gdf.join(df, how='inner')

sdf.sample(5)[[k_var,d_var,s_var]]

In [ ]:
from pysal.contrib.viz import mapping as maps

# Where will our shapefile be stored
shp_link = os.path.join('data','lsoas.shp')

# Save it!
sdf.to_file(shp_link)

# And now re-load the values from the DBF file 
# associated with the shapefile.
values = np.array(ps.open(shp_link.replace('.shp','.dbf')).by_col(s_var))

maps.plot_choropleth(shp_link, values, 'unique_values', 
                     title='SOM Cluster Analysis', 
                     savein=os.path.join('outputs', s_var + '.png'), dpi=150, 
                     figsize=(8,6), alpha=0.9
                    )